In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 필요 데이터 로드

In [ ]:
avg_returns = pd.read_excel('C:/Users/82103/Desktop/YONSEI/2024-1/OR확정모델/프로젝트/최종데이터/avg_returns.xlsx')

df = avg_returns.copy()

df['Date'] = pd.to_datetime(df['Date'])

df['Date'] = df['Date'].dt.year

returns = df.iloc[:, 1:].values

returns.shape

# g_i = 총 수익률 정의

g_i = pd.read_excel('C:/Users/82103/Desktop/YONSEI/2024-1/OR확정모델/프로젝트/최종데이터/total_returns.xlsx')

# 1개년 별로 데이터 분리
g_i['Date'] = pd.to_datetime(g_i['Date'])

g_i_2019 = g_i.loc[0:11]
g_i_2020 = g_i.loc[12:23]
g_i_2021 = g_i.loc[24:35]
g_i_2022 = g_i.loc[36:47]
g_i_2023 = g_i.loc[48:59]

# 공분산 계산
cov_2019 = g_i_2019.drop("Date", axis = 1).cov()
cov_2020 = g_i_2020.drop("Date", axis = 1).cov()
cov_2021 = g_i_2021.drop("Date", axis = 1).cov()
cov_2022 = g_i_2022.drop("Date", axis = 1).cov()
cov_2023 = g_i_2023.drop("Date", axis = 1).cov()

# 시그마_P 정의
def sigma(*args, year):
    weights = []
    for i in args:
        weights.append(i)
    weights = np.array(weights)
    if year == 2019:
        risk = np.sqrt(np.dot(weights.T, np.dot(cov_2019, weights)))
    elif year == 2020:
        risk = np.sqrt(np.dot(weights.T, np.dot(cov_2020, weights)))
    elif year == 2021:
        risk = np.sqrt(np.dot(weights.T, np.dot(cov_2021, weights)))
    elif year == 2022:
        risk = np.sqrt(np.dot(weights.T, np.dot(cov_2022, weights)))
    elif year == 2023:
        risk = np.sqrt(np.dot(weights.T, np.dot(cov_2023, weights)))
    else:
        print("\"연도 입력이 잘못되었습니다. 2019 - 2023 년 사이의 연도 입력\"")
        risk = "error"
    return(risk)

def R_P(weights , returns , year):
    portfolio_returns = returns @ weights

    # 'Date'와 'R_p'를 포함한 결과 데이터프레임 생성
    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p_year =result_df['R_p'][year-2019]
       
    return R_p_year


In [ ]:
# 샤프 지수 계산 함수
def sharpe_ratio(weights, returns, year, R_f):
    R_p = R_P(weights, returns, year)
    sigma_p = sigma(*weights, year = year)
    sharpe = (R_p - R_f) / sigma_p
    return sharpe

In [ ]:
R_f = 0.01

샤프지수 함수 예시

In [ ]:
random_data = np.random.rand(40)
weights = random_data/random_data.sum()

print(sharpe_ratio(weights, returns, 2019, R_f))
print(sharpe_ratio(weights, returns, 2020, R_f))
print(sharpe_ratio(weights, returns, 2021, R_f))
print(sharpe_ratio(weights, returns, 2022, R_f))
print(sharpe_ratio(weights, returns, 2023, R_f))

솔버 만들기

In [ ]:
import cvxpy as cp

2019년

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd


2019년

In [ ]:
def sharpe_ratio(weights):
    portfolio_returns = returns @ weights

    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p = result_df['R_p'][0]
    
    sigma_p = np.sqrt(np.dot(weights.T, np.dot(cov_2019, weights)))

    sharpe = (R_p - 0.01) / sigma_p    
        
    # 확인을 위한 출력
    print(f"Weights: {weights}")
    print(f"R_p: {R_p}")
    print(f"Portfolio Standard Deviation: {sigma_p}")
    print(f"Sharpe Ratio: {sharpe}")
    
    return -sharpe

# 초기 가중치 설정
init_weights = np.ones(40) / 40

# 제약 조건 정의
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# 경계 조건 설정
bounds = tuple((0, 1) for _ in range(40))

# 최적화 문제 해결 (trust-constr 방법 사용)
result = minimize(sharpe_ratio, init_weights, method='trust-constr', bounds=bounds, constraints=constraints)

# 결과 해석
if result.success:
    optimal_weights = result.x
    print("Optimal Weights:", optimal_weights)
    print("Maximum Portfolio Sharpe ratio:", -sharpe_ratio(optimal_weights))
else:
    print("Optimization failed:", result.message)

print(type(optimal_weights))
print(np.sum(optimal_weights))
weights = optimal_weights

In [ ]:
labels = [f'Asset {i+1}' if weight > 0 else '' for i, weight in enumerate(weights)]

# 0이 아닌 값들만 시각화
filtered_weights = [weight for weight in weights if weight > 0]
filtered_labels = [label for label in labels if label != '']

# 도넛 차트 생성
fig, ax = plt.subplots()
ax.pie(filtered_weights, labels=filtered_labels, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.3})

# 원의 중심을 비워서 도넛 차트처럼 보이게 함
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig.gca().add_artist(centre_circle)

# 차트 제목 설정
ax.set_title('2019 Portfolio Distribution')

# 차트 보여주기
plt.show()

weights_2d = weights.reshape(1, -1)
# 히트맵 생성
plt.figure(figsize=(12, 2))
sns.heatmap(weights_2d, cmap='RdBu', vmin=weights.min(), vmax=weights.max(), annot=True, cbar=True)
plt.title('Heatmap of Weights (1D)')
plt.xlabel('Index')
plt.yticks([])  # Y축 눈금 제거
plt.show()

# 1D 배열을 DataFrame으로 변환하여 막대 그래프 생성
weights_df = pd.DataFrame(weights, columns=['Weights'])
weights_df.plot(kind='bar', figsize=(12, 6))
plt.title('Bar Plot of Weights')
plt.xlabel('Index')
plt.ylabel('Weight Value')
plt.show()

2020년

In [ ]:
def sharpe_ratio(weights):
    portfolio_returns = returns @ weights

    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p = result_df['R_p'][1]
    
    sigma_p = np.sqrt(np.dot(weights.T, np.dot(cov_2020, weights)))

    sharpe = (R_p - 0.01) / sigma_p    
        
    # 확인을 위한 출력
    print(f"Weights: {weights}")
    print(f"R_p: {R_p}")
    print(f"Portfolio Standard Deviation: {sigma_p}")
    print(f"Sharpe Ratio: {sharpe}")
    
    return -sharpe

# 초기 가중치 설정
init_weights = np.ones(40) / 40

# 제약 조건 정의
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# 경계 조건 설정
bounds = tuple((0, 1) for _ in range(40))

# 최적화 문제 해결 (trust-constr 방법 사용)
result = minimize(sharpe_ratio, init_weights, method='trust-constr', bounds=bounds, constraints=constraints)

# 결과 해석
if result.success:
    optimal_weights = result.x
    print("Optimal Weights:", optimal_weights)
    print("Maximum Portfolio Sharpe ratio:", -sharpe_ratio(optimal_weights))
else:
    print("Optimization failed:", result.message)

print(type(optimal_weights))
print(np.sum(optimal_weights))
weights = optimal_weights

6분 동안 돌렸는데도 최적해를 찾지 못해서 Solver의 다른 method 이용


저 method의 계산 횟수를 options변수로 추가해서 최적해를 낼 때까지 계속 돌릴 수도 있을 것 같습니다

In [ ]:
def sharpe_ratio(weights):
    portfolio_returns = returns @ weights

    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p = result_df['R_p'][1]
    
    sigma_p = np.sqrt(np.dot(weights.T, np.dot(cov_2020, weights)))

    sharpe = (R_p - 0.01) / sigma_p
    return -sharpe

# 초기 가중치 설정
init_weights = np.ones(40) / 40

# 제약 조건 정의
# 'eq'가 'fun'함수가 0이 되어야 한다는 것을 의미하고 
# 'fun' 함수는 x의 전체 합 - 1로 지정하였다.
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# 결국 각 의사결정변수의 합이 1이 되도록 하는 제약

# 경계 조건 설정
# 각 의사결정변수가 0과 1 사이에 있도록 설정
bounds = tuple((0, 1) for _ in range(40))

# 최적화 문제 해결
result = minimize(sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# 결과 해석
if result.success:
    optimal_weights = result.x
    print("Optimal Weights:", optimal_weights)
    print("Maximum Portfolio Sharpe ratio:", -sharpe_ratio(optimal_weights))
else:
    print("Optimization failed.")

print(type(optimal_weights))
print(np.sum(optimal_weights))
weights = optimal_weights

In [ ]:
labels = [f'Asset {i+1}' if weight > 0 else '' for i, weight in enumerate(weights)]

# 0이 아닌 값들만 시각화
filtered_weights = [weight for weight in weights if weight > 0]
filtered_labels = [label for label in labels if label != '']

# 도넛 차트 생성
fig, ax = plt.subplots()
ax.pie(filtered_weights, labels=filtered_labels, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.3})

# 원의 중심을 비워서 도넛 차트처럼 보이게 함
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig.gca().add_artist(centre_circle)

# 차트 제목 설정
ax.set_title('2020 Portfolio Distribution')

# 차트 보여주기
plt.show()

weights_2d = weights.reshape(1, -1)
# 히트맵 생성
plt.figure(figsize=(12, 2))
sns.heatmap(weights_2d, cmap='RdBu', vmin=weights.min(), vmax=weights.max(), annot=True, cbar=True)
plt.title('Heatmap of Weights (1D)')
plt.xlabel('Index')
plt.yticks([])  # Y축 눈금 제거
plt.show()

# 1D 배열을 DataFrame으로 변환하여 막대 그래프 생성
weights_df = pd.DataFrame(weights, columns=['Weights'])
weights_df.plot(kind='bar', figsize=(12, 6))
plt.title('Bar Plot of Weights')
plt.xlabel('Index')
plt.ylabel('Weight Value')
plt.show()

2021년

In [ ]:
def sharpe_ratio(weights):
    portfolio_returns = returns @ weights

    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p = result_df['R_p'][2]
    
    sigma_p = np.sqrt(np.dot(weights.T, np.dot(cov_2021, weights)))

    sharpe = (R_p - 0.01) / sigma_p
    return -sharpe

# 초기 가중치 설정
init_weights = np.ones(40) / 40

# 제약 조건 정의
# 'eq'가 'fun'함수가 0이 되어야 한다는 것을 의미하고 
# 'fun' 함수는 x의 전체 합 - 1로 지정하였다.
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# 결국 각 의사결정변수의 합이 1이 되도록 하는 제약

# 경계 조건 설정
# 각 의사결정변수가 0과 1 사이에 있도록 설정
bounds = tuple((0, 1) for _ in range(40))

# 최적화 문제 해결
result = minimize(sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# 결과 해석
if result.success:
    optimal_weights = result.x
    print("Optimal Weights:", optimal_weights)
    print("Maximum Portfolio Sharpe ratio:", -sharpe_ratio(optimal_weights))
else:
    print("Optimization failed.")

print(type(optimal_weights))
print(np.sum(optimal_weights))
weights = optimal_weights

In [ ]:
labels = [f'Asset {i+1}' if weight > 0 else '' for i, weight in enumerate(weights)]

# 0이 아닌 값들만 시각화
filtered_weights = [weight for weight in weights if weight > 0]
filtered_labels = [label for label in labels if label != '']

# 도넛 차트 생성
fig, ax = plt.subplots()
ax.pie(filtered_weights, labels=filtered_labels, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.3})

# 원의 중심을 비워서 도넛 차트처럼 보이게 함
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig.gca().add_artist(centre_circle)

# 차트 제목 설정
ax.set_title('2021 Portfolio Distribution')

# 차트 보여주기
plt.show()

weights_2d = weights.reshape(1, -1)
# 히트맵 생성
plt.figure(figsize=(12, 2))
sns.heatmap(weights_2d, cmap='RdBu', vmin=weights.min(), vmax=weights.max(), annot=True, cbar=True)
plt.title('Heatmap of Weights (1D)')
plt.xlabel('Index')
plt.yticks([])  # Y축 눈금 제거
plt.show()

# 1D 배열을 DataFrame으로 변환하여 막대 그래프 생성
weights_df = pd.DataFrame(weights, columns=['Weights'])
weights_df.plot(kind='bar', figsize=(12, 6))
plt.title('Bar Plot of Weights')
plt.xlabel('Index')
plt.ylabel('Weight Value')
plt.show()

2022년

In [ ]:
def sharpe_ratio(weights):
    portfolio_returns = returns @ weights

    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p = result_df['R_p'][3]
    
    sigma_p = np.sqrt(np.dot(weights.T, np.dot(cov_2022, weights)))

    sharpe = (R_p - 0.01) / sigma_p
    return -sharpe

# 초기 가중치 설정
init_weights = np.ones(40) / 40

# 제약 조건 정의
# 'eq'가 'fun'함수가 0이 되어야 한다는 것을 의미하고 
# 'fun' 함수는 x의 전체 합 - 1로 지정하였다.
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# 결국 각 의사결정변수의 합이 1이 되도록 하는 제약

# 경계 조건 설정
# 각 의사결정변수가 0과 1 사이에 있도록 설정
bounds = tuple((0, 1) for _ in range(40))

# 최적화 문제 해결
result = minimize(sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# 결과 해석
if result.success:
    optimal_weights = result.x
    print("Optimal Weights:", optimal_weights)
    print("Maximum Portfolio Sharpe ratio:", -sharpe_ratio(optimal_weights))
else:
    print("Optimization failed.")

print(type(optimal_weights))
print(np.sum(optimal_weights))
weights = optimal_weights

In [ ]:
labels = [f'Asset {i+1}' if weight > 0 else '' for i, weight in enumerate(weights)]

# 0이 아닌 값들만 시각화
filtered_weights = [weight for weight in weights if weight > 0]
filtered_labels = [label for label in labels if label != '']

# 도넛 차트 생성
fig, ax = plt.subplots()
ax.pie(filtered_weights, labels=filtered_labels, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.3})

# 원의 중심을 비워서 도넛 차트처럼 보이게 함
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig.gca().add_artist(centre_circle)

# 차트 제목 설정
ax.set_title('2022 Portfolio Distribution')

# 차트 보여주기
plt.show()

weights_2d = weights.reshape(1, -1)
# 히트맵 생성
plt.figure(figsize=(12, 2))
sns.heatmap(weights_2d, cmap='RdBu', vmin=weights.min(), vmax=weights.max(), annot=True, cbar=True)
plt.title('Heatmap of Weights (1D)')
plt.xlabel('Index')
plt.yticks([])  # Y축 눈금 제거
plt.show()

# 1D 배열을 DataFrame으로 변환하여 막대 그래프 생성
weights_df = pd.DataFrame(weights, columns=['Weights'])
weights_df.plot(kind='bar', figsize=(12, 6))
plt.title('Bar Plot of Weights')
plt.xlabel('Index')
plt.ylabel('Weight Value')
plt.show()

2023년

In [ ]:
def sharpe_ratio(weights):
    portfolio_returns = returns @ weights

    result_df = pd.DataFrame({
        'Date': df['Date'],
        'R_p': portfolio_returns
    })
    R_p = result_df['R_p'][4]
    
    sigma_p = np.sqrt(np.dot(weights.T, np.dot(cov_2023, weights)))

    sharpe = (R_p - 0.01) / sigma_p
    return -sharpe

# 초기 가중치 설정
init_weights = np.ones(40) / 40

# 제약 조건 정의
# 'eq'가 'fun'함수가 0이 되어야 한다는 것을 의미하고 
# 'fun' 함수는 x의 전체 합 - 1로 지정하였다.
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# 결국 각 의사결정변수의 합이 1이 되도록 하는 제약

# 경계 조건 설정
# 각 의사결정변수가 0과 1 사이에 있도록 설정
bounds = tuple((0, 1) for _ in range(40))

# 최적화 문제 해결
result = minimize(sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# 결과 해석
if result.success:
    optimal_weights = result.x
    print("Optimal Weights:", optimal_weights)
    print("Maximum Portfolio Sharpe ratio:", -sharpe_ratio(optimal_weights))
else:
    print("Optimization failed.")

print(type(optimal_weights))
print(np.sum(optimal_weights))
weights = optimal_weights

In [ ]:
labels = [f'Asset {i+1}' if weight > 0 else '' for i, weight in enumerate(weights)]

# 0이 아닌 값들만 시각화
filtered_weights = [weight for weight in weights if weight > 0]
filtered_labels = [label for label in labels if label != '']

# 도넛 차트 생성
fig, ax = plt.subplots()
ax.pie(filtered_weights, labels=filtered_labels, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.3})

# 원의 중심을 비워서 도넛 차트처럼 보이게 함
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig.gca().add_artist(centre_circle)

# 차트 제목 설정
ax.set_title('2023 Portfolio Distribution')

# 차트 보여주기
plt.show()

weights_2d = weights.reshape(1, -1)
# 히트맵 생성
plt.figure(figsize=(12, 2))
sns.heatmap(weights_2d, cmap='RdBu', vmin=weights.min(), vmax=weights.max(), annot=True, cbar=True)
plt.title('Heatmap of Weights (1D)')
plt.xlabel('Index')
plt.yticks([])  # Y축 눈금 제거
plt.show()

# 1D 배열을 DataFrame으로 변환하여 막대 그래프 생성
weights_df = pd.DataFrame(weights, columns=['Weights'])
weights_df.plot(kind='bar', figsize=(12, 6))
plt.title('Bar Plot of Weights')
plt.xlabel('Index')
plt.ylabel('Weight Value')
plt.show()